# Kālī

Kālī is a software package written to model and study determinstic and stochastic astronomical light curves, i.e. ordered lists of flux as a function of time. Kālī provides two models for astronomical light curves: 
1. Continuous-time Autoregressive Moving Average (C-ARMA) processes (stochastic).
2. Binary Supermassive Black Holes (bSMBH) with relativistic doppler beaming (determinsistic).

## Concepts

Kālī consists of multiple Python sub-modules that call underlying c++ libraries using Cython. All the sub-modules are structured around two concepts: `lc` objects that represent light curves and `task` objects that take `lc` objects as inputs and perform useful work such as fitting light curves to various stochastic models. Both the `lc` and `task` objects are abstract objects that must be inherited from to make concrete instantiable sub-classes such as the `basicLC`, `externalLC`, and the `basicTask` objects. The abstract `lc` and `task` classes as well as the concrete `basicLC`, `externalLC`, and `basicTask` objects are all defined in the `libcarma` sub-module.

The two concrete classes `basicLC` and `externalLC` are designed for slightly different purposes. `basicLC`s are to be used for simulated data generated by various `task` objects while `externalLC`s are to be used when performing one-off analysis of a real light curve. Typically, if multiple real light curves are to be studied with Kālī, it is better to define custom `lc` classes such as the `sdssLC` defined in `sdss.py` and the `k2LC` defined in the `k2.py` sub-modules. Although the provided `sdssLC` and `k2LC` are trivial classes that read the sample data provided in the `examples/data` folder, more serious implementations of such classes can be sophisticated enough to go automatically fetch remotely hosted data, pre-process the data, determine measurement uncertainties from the observations etc... The idea is to let data-set specific sub-classes of `lc` specialize for the data-set being used and make the remaining package agnostic to the source of the data. Later on in this document, we will provide guidelines for creating inherited sub-classes of `lc`.

We shall begin by introducing the reader to the `basicTask` object.

## `basicTask`s and `basicLC`s

The `basicTask` object allows us to perform basic C-ARMA analysis on light curves. In order to use a `basicTask`, it is necessary to import the `libcarma` sub-module. After this, we can create a new basic task object called `newTask` to study the C-ARMA(2,1) model as follows:

In [1]:
import libcarma
newTask = libcarma.basicTask(2,1)

All `task` objects contain one or more models i.e. each task object is capable of being used to study multiple models simultaneously. The only restriction is that the models held by each task object have to have the order i.e. in the case of `basicTask`s that are designed for C-ARMA models, all the C-ARMA models held by a single `basicTask` have to have the the same `p` & `q` order. Of course, this does not prevent us from having multiple task objects declared at once. By default, the `task` object initializer will query your system to determine how many hardware threads (including Intel Hyperthreading) your CPU supports and will hold that number of models. However, this behavior can be changed by explicitly instructing the task constructor to create a specific number of models in the task as follows:

In [2]:
anotherNewTask = libcarma.basicTask(3, 1, nthreads = 2)

`anotherNewTask` is a `basicTask` that holds two C-ARMA(3,1) models for use. In the case of `basicTask`s, we can additionally specify how models will burn in light curves when generating simulated light curves by initializing the `yetAnotherNewTask` object as follows:

In [3]:
yetAnotherNewTask = libcarma.basicTask(4, 2, nthreads = 1, nburn = 100000)

`yetAnotherNewTask` is a `basicTask` that holds a single C-ARMA(4,2) model. Any simulated light curves generated by `yetAnotherNewTask` will automatically be burnt in for 100,000 steps. Other parameters that can be passed to the `basicTask` constructor include `nwalkers`, `nsteps` etc... Please refer to the code documentation for the exact list of parameters accepted by the constructor.

At this point, our `basicTask` objects are hollow objects that lack numerical meaning. We must specify C-ARMA model parameters to be able to do useful things with our tasks. Two model parameters are required - a double precision value for the time-step `dt` to be used for simulations and a numpy ndarray of doubles `Theta` for the C-ARMA model coefficients. Mathematically, an individual C-ARMA model can be specified using either the co-efficients representation or the roots representation. The roots representation is much more convenient to use when providing model parametrs because it is trivial to pick valid values. However, Kālī requires that the C-ARMA model be specified purely in terms of the C-ARMA model coefficients and provides helper functions to convert the roots representation to the coeffiecients represenation. Let us set the first C-ARMA model in the `newTask` object to have an autocorrelation function that decays with two timescales - `114` d and `43.2` d. We shall set the MA root to be `0.4` and set the amplitude of the light curve (i.e. the square root of the asymptotic autocorrelation function) to be 1.0

In [4]:
r_1 = (1.0/114.0) + 0j
r_2 = (-1.0/43.2) + 0j
m_1 = -0.4 + 0j
amp = 1.0

Note that we have purposely made the first root positive i.e. the C-ARMA model corresponding to this set of roots will be unstable. We can use the helper function `libcarma.coeffs(p, q, Rho)` to compute the polynomial coefficients representation corresponding to this roots representation as follows: 

In [5]:
import numpy as np
Rho = np.array([r_1, r_2, m_1, amp])
Theta = libcarma.coeffs(2, 1, Rho)
print Theta

[  1.43762183e-02  -2.03053931e-04   4.00000000e-01   1.00000000e+00]


We can check the returned `Theta` for validity using the `check(Theta)` method of the `basicTask` object `newTask` as follows:

In [6]:
print newTask.check(Theta)

False


After fixing the bad root, we can rerun the check as follows

In [7]:
r_1 = (-1.0/114.0) + 0j
Rho = np.require(np.array([r_1, r_2, m_1, amp]), requirements=['F', 'A', 'W', 'O', 'E'])
Theta = libcarma.coeffs(2, 1, Rho)
print Theta
print newTask.check(Theta)

[ 0.03192008  0.00020305  0.00359813  0.00899533]
True


Note that we wrapped the definition of Rho in a `np.require` call that ensures that the underlying data in `Rho` is contiguous in memory and can thus be accesses by simple pointer arithmetic in the underlying `c++` library. Although it is not necesary to wrap such small arrays in the `np.require` call, it is good practice and can help avoid mysterious memeory related errors as well as silent garbage output caused by an incorrect interpretation of a pointer as data. 

After having established that we have a good C-ARMA model coefficients vector, let us set the first C-ARMA model in `newTask` to use this vector. We also pick the time increment `dt` between points in the light curve to be `dt = 0.1` d. We can set the first C-ARMA model held by `newTask` to the specified `dt` and `Theta` as follows:

In [8]:
dt = 0.1
newTask.set(dt, Theta, tnum = 0)

0

The return value of `0` indicates that the method call completed without error. We can check to see which of the C-ARMA models held by `newTask` has been set as follows:

In [9]:
newTask.list()

array([ True, False, False, False, False, False, False, False], dtype=bool)

As can be seen above, only the first model has been set with values so far. We can see the resulting values of the C-ARMA model matrices as follows:

In [10]:
newTask.show()

As can be seen by the way we issued the two previous commands, we do not have to specify `tnum = 0` because the default for `tnum` is `0` i.e. the first C-ARMA model held by the task. `tnum` was not required for the call to `set(dt, Theta)` either. Having set the model, we can also obtain the asymptotic standard deviation of light curves made with this model, i.e. the variability of the light curve, as follows:

In [11]:
import math
math.sqrt(newTask.Sigma()[0,0])

1.0

It turns out that the variability amplitude is `1.0` flux units which should come as no surprise given that we picked `amp` in the roots representation `Rho` to be `1.0` to begin with. At this point, we are ready to simulate a light curve using this C-ARMA model. The `simulate(T)` method of `basictask` requires the duration of the desired light curve and returns a `basicLC` object. We can create a new light curve of length `500` d as follows:

In [12]:
newLC = newTask.simulate(500.0)

We can view the simulated light curve `newLC` using the `plot()` method of all `lc` objects as follows:

In [13]:
import matplotlib.pyplot as plt
plt.ion()
newLC.plot()
plt.show()
#plt.savefig('lc_noNoise.jpg', dpi = 1200)

/home/vish/anaconda2/lib/python2.7/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


<img src="lc_noNoise.jpg">

Now this light curve has no noise. We can simulate noise by fixing two parameters - the fractional level of variability (fracIntrinsicVar = 0.15 by default) and the fractional level of the noise to the signal (fracNoiseToSignal = 0.001 by default). Once we have fixed these two parameters, we can simulate noise as follows:

In [14]:
newTask.observe(newLC)

Once again, we can plot the resulting light curve using the `plot()` method of all `lc` objects as follows:

In [15]:
newLC.plot()
plt.show()
#plt.savefig('lc_Noise.jpg', dpi = 1200)

<img src="lc_Noise.jpg">

Consider the effect of the noise. A high order C-ARMA(p,q) model with large p but small q tends to be fairly smooth. Gaussian white noise can potentially make the light curve look less smooth than it actually is! We can compute the log likelihood of this light curve under the same C-ARMA model that it was computed using as follows: 

In [16]:
print newTask.logLikelihood(newLC)

17091.7193485


In a Bayesian context, we can assign a prior to the C-ARMA parameter values based on factors such as the length of the light curve and the median/mean seperation between observations etc... For example, C-ARMA model parameters with inbuilt timescales that are of significantly longer duration than the length of the observed light curve must be treated with suspicion. Note that we have to be very careful here with how we think about this. It is entirely possible to observe a light curve for too short a duration - that is a failing of the observations. In such cases, it is impossible for us to get an accurate estimate of the true timescale from such observations. But if the algorithm suggests that the actual timescale is many times longer than the duration of our observations, we must treat the result with suspicion. We can compute the log pior probability of a model given a light curve as follows:

In [17]:
print newTask.logPrior(newLC)

-inf


Since this light curve is a litte over twice as long as the longest in-built timescale (about 65 days), and since the predicted variance of the light curve matches well with the variance of the light curve, etc..., the log prior is 0.0 indicating that the model is acceptable. Given the likelihood and the prior, we can compute the posterior probability using simple addition:

In [18]:
print newTask.logPosterior(newLC)

-inf


How do we control the prior? The lightcurve object has three variables that are CRITICAL! - the maxSigma, the minTimescale, and the maxTimescale. The default values are conservative - maxSigma = 2.0, minTimescale  = 2.0 and maxTimescale = 0.5. How are these values used? If the sqrt(Sigma[0,0]) i.e. the theoretical asymptotic variance of the light curve is greater than maxSigma time the standard devaition of the observed light curve, we set the prior to 0 i.e the logPrior to -infinity. If any timescale corresponding to the AR coefficients is greater than maxTimescale times the duration of the observed light curve or if it is shorter than minTimescale time the smallest timestep, we set the prior to 0. This way, we can insist on the code returning sane results. If the light curve is very short, it may be helpful to turn maxSigma and maxTimescale up a bit. If the light curve is very poorly sampled, it may help to turn minTimescale down. However, loosening the priors too much can result in spurious peaks of the likelihood space being returned as optimal. Generally speaking, it is better to err on the tighter side until we have more experience with C-ARMA modelling. 